![Photo by Stephen Phillips - Hostreviews.co.uk on UnSplash](https://cf.bstatic.com/xdata/images/hotel/max1024x768/408003083.jpg?k=c49b5c4a2346b3ab002b9d1b22dbfb596cee523b53abef2550d0c92d0faf2d8b&o=&hp=1){fig-align="center" width=50%}


# Import data

In [1]:
import time
from pathlib import Path

import lightgbm as lgb
import pandas as pd
from catboost import CatBoostRegressor
from data import utils
from lets_plot import *
from lets_plot.mapping import as_discrete
from sklearn import compose, impute, model_selection, pipeline, preprocessing
from xgboost import XGBRegressor

LetsPlot.setup_html()

**Objective**:
* Examine the necessary sample pre-processing steps before modeling
* 
Create the required pipeline
* 
Evaluate multiple algorith
* .
Choose a suitable baseline moel.





# Train-test split

In [5]:
utils.seed_everything(utils.Configuration.seed)

df = pd.read_parquet(
    utils.Configuration.INTERIM_DATA_PATH.joinpath(
        "2023-10-01_Processed_dataset_for_NB_use.parquet.gzip"
    )
)

df = df.sample(frac=1, random_state=utils.Configuration.seed).reset_index(drop=True)


X = df.drop(columns=utils.Configuration.target_col)
y = df[utils.Configuration.target_col]

In [14]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=0.2, random_state=utils.Configuration.seed
)